In [1]:
'''
本文件用于生成BPP问题数据集，实现参考了algorithms1，但目前只是一个初步的实现，仅生成一个数据
等确定了task1和task2的模型输入要求后，更改以生成匹配的数据集格式
       Z
       |
       |
       |________Y
      /
     /
    X
'''


import random
import csv

def generate_bpp_dataset(
    initial_item=(100, 100, 100),
    N_range=(10, 50),
    output_file="bpp_dataset.csv"
):
    """
    生成BPP问题数据集
    initial_item: 初始物品尺寸 (l, w, h)
    N_range: N的取值范围(min, max)
    output_file: 数据输出文件
    """
    # 随机选择N
    N = random.randint(N_range[0], N_range[1])
    
    # I初始化为包含一个大立方体
    # 对于每个物品，我们存储：(l, w, h, pos_x, pos_y, pos_z, rot_x, rot_y, rot_z)
    # pos_x, pos_y, pos_z为在容器坐标系下的相对位置(设定为物品到原点距离最近的端点)
    # rot_x, rot_y, rot_z为旋转信息(这里仅简化为标记物品是否旋转了其轴向维度)
    I = [
        {
            "l": initial_item[0],   # 物品长
            "w": initial_item[1],   # 物品宽
            "h": initial_item[2],   # 物品高
            "pos_x": 0.0,          # 物品在容器坐标系下的x坐标
            "pos_y": 0.0,          # 物品在容器坐标系下的y坐标
            "pos_z": 0.0,          # 物品在容器坐标系下的z坐标
            "rot_x": initial_item[0],   # 物品实际堆叠时的长
            "rot_y": initial_item[1],   # 物品实际堆叠时的宽
            "rot_z": initial_item[2]    # 物品实际堆叠时的高
        }
    ]
    
    # 在达到N之前持续分裂物品
    while len(I) < N:
        # 按体积加权从I中选取物品
        volumes = [item["rot_x"] * item["rot_y"] * item["rot_z"] for item in I]
        total_volume = sum(volumes)
        r = random.random() * total_volume
        cumulative = 0.0
        chosen_index = None
        for idx, vol in enumerate(volumes):
            cumulative += vol
            if r <= cumulative:
                chosen_index = idx
                break
        
        # 选中的物品
        chosen_item = I.pop(chosen_index)
        
        # 根据边长选择分割轴(以边长为权重)
        edges = [chosen_item["rot_x"], chosen_item["rot_y"], chosen_item["rot_z"]]
        edge_sum = sum(edges)
        r_edge = random.random() * edge_sum
        cumulative_edge = 0.0
        axis = None  # 0->x, 1->y, 2->z
        for i, e_len in enumerate(edges):
            cumulative_edge += e_len
            if r_edge <= cumulative_edge:
                axis = i
                break
        
        # 沿所选轴方向的中点附近随机选择分割位置
        # 为了保证不会出现太极端的值，以物品一半长度为参考，在中点附近±1/4范围随机选取
        axis_length = edges[axis]
        half_axis_length = axis_length / 2.0
        split_offset = (random.random() - 0.5) * (axis_length / 2.0)
        split_pos1 = half_axis_length + split_offset
        split_pos2 = half_axis_length - split_offset
        
        # 将物品分割为两个子物品
        if axis == 0:
            child1 = {"l": 0, "w": 0, "h": 0, "pos_x": chosen_item["pos_x"], "pos_y": chosen_item["pos_y"], "pos_z": chosen_item["pos_z"],"rot_x": split_pos1,"rot_y": chosen_item["rot_y"],"rot_z": chosen_item["rot_z"]}
            child2 = {"l": 0, "w": 0, "h": 0, "pos_x": chosen_item["pos_x"]+split_pos1, "pos_y": chosen_item["pos_y"], "pos_z": chosen_item["pos_z"],"rot_x": split_pos2,"rot_y": chosen_item["rot_y"],"rot_z": chosen_item["rot_z"]}
        if axis == 1:
            child1 = {"l": 0, "w": 0, "h": 0, "pos_x": chosen_item["pos_x"], "pos_y": chosen_item["pos_y"], "pos_z": chosen_item["pos_z"],"rot_x": chosen_item["rot_x"],"rot_y": split_pos1,"rot_z": chosen_item["rot_z"]}
            child2 = {"l": 0, "w": 0, "h": 0, "pos_x": chosen_item["pos_x"], "pos_y": chosen_item["pos_y"]+split_pos1, "pos_z": chosen_item["pos_z"],"rot_x": chosen_item["rot_x"],"rot_y": split_pos2,"rot_z": chosen_item["rot_z"]}
        if axis == 2:
            child1 = {"l": 0, "w": 0, "h": 0, "pos_x": chosen_item["pos_x"], "pos_y": chosen_item["pos_y"], "pos_z": chosen_item["pos_z"],"rot_x": chosen_item["rot_x"],"rot_y": chosen_item["rot_y"],"rot_z": split_pos1}
            child2 = {"l": 0, "w": 0, "h": 0, "pos_x": chosen_item["pos_x"], "pos_y": chosen_item["pos_y"], "pos_z": chosen_item["pos_z"]+split_pos1,"rot_x": chosen_item["rot_x"],"rot_y": chosen_item["rot_y"],"rot_z": split_pos2}
        
        # 旋转子物品
        def rotate_item(l, w, h):
            # 随机旋转操作：随机打乱(l,w,h)顺序
            dims = [l, w, h]
            random.shuffle(dims)
            return dims[0], dims[1], dims[2]
        
        child1["l"], child1["w"], child1["h"] = rotate_item(child1["rot_x"], child1["rot_y"], child1["rot_z"])
        child2["l"], child2["w"], child2["h"] = rotate_item(child2["rot_x"], child2["rot_y"], child2["rot_z"])

        # 加入到I中
        I.append(child1)
        I.append(child2)

    # 当I长度达到N时写入文件
    fieldnames = ["l", "w", "h", "pos_x", "pos_y", "pos_z", "rot_x", "rot_y", "rot_z"]
    with open(output_file, "w", newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for item in I:
            writer.writerow(item)

    return I

if __name__ == "__main__":
    generated_items = generate_bpp_dataset(output_file="./Dataset/Task1&2/task1.csv")
    print("Generated dataset saved. Items count:", len(generated_items))


Generated dataset saved. Items count: 28
